In [1]:
import tensorflow as tf
import pandas as pl
import numpy as np
import IPython

from window_generator import WindowGenerator

2023-03-28 17:03:08.476812: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-28 17:03:08.512756: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-28 17:03:08.513162: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 17:03:09.236662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pl.read_csv("../data/F_prices.csv")

# clean data
# df.drop_in_place('Date')
df = df.drop(columns='Date')

# split data
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

In [3]:
print(train_df, val_df, test_df)

         Price    Volume
0     0.251450   1091238
1     0.250977   1174468
2     0.250977   5209582
3     0.248135   1424158
4     0.246715    675088
...        ...       ...
8966  4.282694  51625000
8967  4.337996  51345200
8968  4.294983  52307700
8969  4.233540  39308200
8970  4.178239  32642300

[8971 rows x 2 columns]           Price    Volume
8971   4.184383  43893100
8972   4.153661  30154300
8973   4.172093  29402600
8974   4.172093  38574200
8975   4.147517  12102200
...         ...       ...
11529  8.596539  28208300
11530  8.476699  41041800
11531  8.492680  27617900
11532  8.468709  36037700
11533  8.492680  25409600

[2563 rows x 2 columns]            Price    Volume
11534   8.548603  21786800
11535   8.700401  27157800
11536   8.476699  43502000
11537   8.476699  41321400
11538   8.221042  75841400
...          ...       ...
12811  11.480000  69729100
12812  11.420000  74225300
12813  11.510000  72425300
12814  11.520000  55899200
12815  11.580000  28695779

[1282 rows x 

In [4]:
window = WindowGenerator(input_width=10, label_width=10, shift=1, label_columns=['Price'], train_df=train_df, val_df=val_df, test_df=test_df)

window

Total window size: 11
Input indices: [0 1 2 3 4 5 6 7 8 9]
Label indices: [ 1  2  3  4  5  6  7  8  9 10]
Label column name(s): ['Price']

In [5]:
example_window = tf.stack([np.array(train_df[:window.total_window_size]),
                           np.array(train_df[100:100+window.total_window_size]),
                           np.array(train_df[200:200+window.total_window_size])])

example_inputs, example_labels = window.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')


All shapes are: (batch, time, features)
Window shape: (3, 11, 2)
Inputs shape: (3, 10, 2)
Labels shape: (3, 10, 1)


2023-03-28 17:03:10.207539: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-03-28 17:03:10.207568: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: gone
2023-03-28 17:03:10.207574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: gone
2023-03-28 17:03:10.207640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 515.86.1
2023-03-28 17:03:10.207658: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 515.86.1
2023-03-28 17:03:10.207663: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 515.86.1


In [6]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(units=1)
])

In [7]:
print('Input shape:', window.example[0].shape)
print('Output shape:', lstm_model(window.example[0]).shape)


2023-03-28 17:03:10.391850: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [8971,2]
	 [[{{node Placeholder/_0}}]]
2023-03-28 17:03:10.392205: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]


Input shape: (32, 10, 2)
Output shape: (32, 10, 1)


2023-03-28 17:03:10.672938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-28 17:03:10.674555: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-28 17:03:10.675959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [8]:
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=2,
                                                  mode='min')

history = lstm_model.fit(window.train, epochs=20,
                         validation_data=window.val,
                         callbacks=[])

Epoch 1/20


2023-03-28 17:03:10.905203: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]
2023-03-28 17:03:10.905552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]
2023-03-28 17:03:11.083108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node grad

277/281 [============================>.] - ETA: 0s - loss: 29.1138 - mean_absolute_error: 3.6562

2023-03-28 17:03:13.706587: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype int32
	 [[{{node Placeholder/_14}}]]
2023-03-28 17:03:13.707011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int32
	 [[{{node Placeholder/_13}}]]
2023-03-28 17:03:13.890379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node grad

281/281 [==============================] - 3s 6ms/step - loss: 29.1209 - mean_absolute_error: 3.6556 - val_loss: 56.3420 - val_mean_absolute_error: 7.0228
Epoch 2/20
281/281 [==============================] - 1s 5ms/step - loss: 21.4802 - mean_absolute_error: 3.1097 - val_loss: 41.9462 - val_mean_absolute_error: 5.9601
Epoch 3/20
281/281 [==============================] - 1s 5ms/step - loss: 17.9951 - mean_absolute_error: 2.9922 - val_loss: 33.2507 - val_mean_absolute_error: 5.2502
Epoch 4/20
281/281 [==============================] - 1s 5ms/step - loss: 16.6323 - mean_absolute_error: 3.0270 - val_loss: 28.3378 - val_mean_absolute_error: 4.8139
Epoch 5/20
281/281 [==============================] - 1s 5ms/step - loss: 16.2465 - mean_absolute_error: 3.0846 - val_loss: 25.9000 - val_mean_absolute_error: 4.5889
Epoch 6/20
281/281 [==============================] - 1s 5ms/step - loss: 16.1583 - mean_absolute_error: 3.1185 - val_loss: 24.8017 - val_mean_absolute_error: 4.4857
Epoch 7/20
281/

In [10]:
IPython.display.clear_output()
val_performance_lstm = lstm_model.evaluate(window.val)
performance_lstm = lstm_model.evaluate(window.test, verbose=0)

77/80 [===========================>..] - ETA: 0s - loss: 23.8941 - mean_absolute_error: 4.3971

2023-03-28 17:03:41.276934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int32
	 [[{{node Placeholder/_13}}]]
2023-03-28 17:03:41.277287: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [2563,2]
	 [[{{node Placeholder/_0}}]]


80/80 [==============================] - 0s 2ms/step - loss: 24.0404 - mean_absolute_error: 4.4137


2023-03-28 17:03:41.524654: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1282,2]
	 [[{{node Placeholder/_0}}]]
2023-03-28 17:03:41.525106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_13' with dtype int32
	 [[{{node Placeholder/_13}}]]


In [11]:
print(val_performance_lstm, performance_lstm)

[24.040401458740234, 4.413681983947754] [52.03739929199219, 6.331363201141357]
